In [ ]:
from torch.utils.data import DataLoader, WeightedRandomSampler
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.data import DataLoader
import wandb
from models.vgg19_model import get_vgg19_model
from models.vit_model import get_vit_model
from models.alexnet_model import get_alexnet_model
from models.gideon_alex_net import get_gideon_alexnet_model
from models.resnet_model import get_resnet_model
from models.resnet_model import get_timm_model
from models.dino_model import get_dinov2_model
from torchvision import transforms
from preprocessing.clahe import CLAHETransform
from utils.logger import init_wandb
from collections import Counter
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from losses.label_smoothing import LabelSmoothingCrossEntropy
from losses.confidence_weighted_loss import ConfidenceWeightedCrossEntropy
from losses.combined_loss import CombinedLabelSmoothingConfidenceWeightedLoss

from utils.hard_sampling import get_low_confidence_samples
from utils.saver import save_test_outputs
import random
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import KFold
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F

from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import KFold
from dataset_handler.dataset import ImageDataset
from preprocessing.clahe import CLAHETransform
import tqdm

ImportError: attempted relative import with no known parent package

In [ ]:
def ensemble_test_predictions(models, test_dataset):
    loader = DataLoader(test_dataset, batch_size=32)
    all_probs = []
    true_labels = []
    filenames = []

    for images, labels, paths in loader:
        images = images.to("cuda")
        batch_probs = []

        for model in models:
            model.eval()
            with torch.no_grad():
                logits = model(images)
                probs = F.softmax(logits, dim=1)
                batch_probs.append(probs.cpu())

        avg_probs = torch.stack(batch_probs).mean(dim=0)
        all_probs.append(avg_probs)
        true_labels.extend(labels.numpy())
        filenames.extend(paths)

    all_probs = torch.cat(all_probs).numpy()
    predicted_labels = np.argmax(all_probs, axis=1)
    return filenames, true_labels, predicted_labels, all_probs